# 🗑️ VLM для распознавания мусора (Ансамбль + LLM)

**Архитектура:**
```
Изображение → Ваш Ансамбль (YOLO+RT-DETR) → Детекции → LLM → Ответ
```

Этот подход использует ваши обученные детекторы как "глаза" VLM.


In [ ]:
# 1. Установка и клонирование
!git lfs install
!git clone https://github.com/sry4theweight/VLM_garbage.git
%cd VLM_garbage

# Установка зависимостей
!pip install -q transformers peft accelerate ultralytics supervision tqdm pillow roboflow

# Проверка GPU
import torch
print(f"PyTorch: {torch.__version__}")
print(f"CUDA: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"GPU: {torch.cuda.get_device_name(0)}")


In [ ]:
# 2. Скачать датасет Roboflow
!python download_roboflow_dataset.py


In [ ]:
# 3. Загрузка моделей
import sys
sys.path.append('.')
import torch
from PIL import Image
import matplotlib.pyplot as plt
import matplotlib.patches as patches
from pathlib import Path
import random
from transformers import BlipProcessor, BlipForConditionalGeneration

# Загрузка ансамбля детекторов (ваши модели)
from vlm_annotation.ensemble_detector import EnsembleDetector

detector = EnsembleDetector(
    yolo_model_path="models/yolo/yolov8x/best.pt",
    detr_model_path="models/rt-detr/rt-detr-101/m",
    detr_processor_path="models/rt-detr/rt-detr-101/p",
    conf_threshold=0.5
)
print("✅ Ансамбль детекторов загружен!")

# Загрузка BLIP для описания сцены
device = "cuda" if torch.cuda.is_available() else "cpu"
blip_processor = BlipProcessor.from_pretrained("Salesforce/blip-image-captioning-large")
blip_model = BlipForConditionalGeneration.from_pretrained(
    "Salesforce/blip-image-captioning-large"
).to(device)
blip_model.eval()
print("✅ BLIP загружен для описания сцены!")


In [ ]:
# 4. Собираем изображения БЕЗ АУГМЕНТАЦИЙ (valid + test)
valid_dir = Path("data/roboflow_dataset/valid")
test_dir = Path("data/roboflow_dataset/test")

all_images = list(valid_dir.glob("*.jpg")) + list(test_dir.glob("*.jpg"))
print(f"✅ Найдено {len(all_images)} изображений (valid + test, без аугментаций)")


In [ ]:
# 5. Класс VLM: Ансамбль + BLIP для полного описания
COLORS = {'glass': 'green', 'plastic': 'red', 'metal': 'cyan', 'paper': 'yellow', 'organic': 'lime'}

def describe_scene(image):
    """Описание сцены с помощью BLIP (трава, улица, песок и т.д.)"""
    inputs = blip_processor(image, return_tensors="pt").to(device)
    with torch.no_grad():
        out = blip_model.generate(**inputs, max_length=50)
    return blip_processor.decode(out[0], skip_special_tokens=True)

def describe_garbage_location(image, detections):
    """Описание где находится мусор"""
    if not detections:
        return "No garbage detected in this image."
    
    # Получаем описание сцены
    scene = describe_scene(image)
    
    # Извлекаем поверхность из описания
    surfaces = ['grass', 'ground', 'sand', 'pavement', 'road', 'floor', 'table', 'street', 'beach', 'soil']
    found_surface = None
    for s in surfaces:
        if s in scene.lower():
            found_surface = s
            break
    
    # Подсчёт объектов
    class_counts = {}
    for det in detections:
        label = det['label']
        class_counts[label] = class_counts.get(label, 0) + 1
    
    # Формируем описание
    items = []
    for label, count in class_counts.items():
        if count == 1:
            items.append(f"a {label} item")
        else:
            items.append(f"{count} {label} items")
    
    garbage_str = ", ".join(items[:-1]) + " and " + items[-1] if len(items) > 1 else items[0]
    
    if found_surface:
        return f"There is {garbage_str} on the {found_surface}."
    else:
        return f"There is {garbage_str} in this image. Scene: {scene}"

def answer_question(image, detections, question):
    """Ответ на вопрос о мусоре"""
    q_lower = question.lower()
    
    # Подсчёт
    class_counts = {}
    for det in detections:
        label = det['label']
        class_counts[label] = class_counts.get(label, 0) + 1
    total = len(detections)
    
    # Вопрос о количестве
    if "how many" in q_lower:
        for cls in ['glass', 'plastic', 'metal', 'paper', 'organic']:
            if cls in q_lower:
                count = class_counts.get(cls, 0)
                return f"There {'is' if count == 1 else 'are'} {count} {cls} object{'s' if count != 1 else ''}."
        return f"There are {total} garbage objects in total."
    
    # Вопрос о наличии
    if "is there" in q_lower or "are there" in q_lower:
        for cls in ['glass', 'plastic', 'metal', 'paper', 'organic']:
            if cls in q_lower:
                count = class_counts.get(cls, 0)
                if count > 0:
                    return f"Yes, there {'is' if count == 1 else 'are'} {count} {cls} object{'s' if count != 1 else ''}."
                else:
                    return f"No, there is no {cls} in this image."
    
    # Вопрос "что на картинке"
    if "what" in q_lower:
        return describe_garbage_location(image, detections)
    
    # По умолчанию - описание
    return describe_garbage_location(image, detections)

def test_vlm_on_image(image_path, questions):
    print(f"\n{'='*60}")
    print(f"📷 {Path(image_path).name}")
    print('='*60)
    
    # Загрузка и детекция
    img = Image.open(image_path).convert('RGB')
    detections = detector.detect(img)
    
    # Визуализация
    fig, ax = plt.subplots(1, figsize=(10, 7))
    ax.imshow(img)
    
    for det in detections:
        bbox = det['box']
        color = COLORS.get(det['label'], 'white')
        rect = patches.Rectangle((bbox[0], bbox[1]), bbox[2]-bbox[0], bbox[3]-bbox[1],
                                  linewidth=2, edgecolor=color, facecolor='none')
        ax.add_patch(rect)
        ax.text(bbox[0], bbox[1]-5, f"{det['label']} {det['confidence']:.0%}", 
                color=color, fontsize=9, weight='bold')
    
    ax.set_title(f"Детекции: {len(detections)} объектов")
    ax.axis('off')
    plt.show()
    
    # Описание сцены
    scene = describe_scene(img)
    print(f"\n🌍 Сцена: {scene}")
    
    # Ответы на вопросы
    print("\n💬 Вопросы и ответы:")
    for q in questions:
        answer = answer_question(img, detections, q)
        print(f"\nQ: {q}")
        print(f"A: {answer}")


In [ ]:
# 6. Тестируем на 3 случайных изображениях
questions = [
    "What is in this image?",  # Что на картинке?
    "Is there any plastic in this image?",
    "Is there any glass?",
    "How many objects are there?",
]

for img_path in random.sample(all_images, min(3, len(all_images))):
    test_vlm_on_image(img_path, questions)


In [ ]:
# 7. Интерактивное тестирование - измените вопрос!
image_path = random.choice(all_images)
my_question = "Is there any glass in this image?"  # <-- Ваш вопрос

test_vlm_on_image(image_path, [my_question])


In [ ]:
# 8. Сохранение на Google Drive
from google.colab import drive
drive.mount('/content/drive')

!mkdir -p /content/drive/MyDrive/VLM_garbage
!cp -r models /content/drive/MyDrive/VLM_garbage/
!cp -r vlm_inference /content/drive/MyDrive/VLM_garbage/
!cp -r vlm_annotation /content/drive/MyDrive/VLM_garbage/

print("✅ Сохранено на Google Drive: /MyDrive/VLM_garbage")
